# Import Needed Packages

In [13]:
import boto3
import sagemaker
from pyathena import connect

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to S3 Bucket

In [3]:
from sagemaker import get_execution_role

# Get execution permissions
iam_role = get_execution_role()

In [93]:
# Specify Bucket Name
bucket_name = "ads-508-spring2023-team3"

# Set up session parameters
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
personal_bucket = sagemaker_session.default_bucket()

# Establish an S3 connection
s3 = boto3.Session().client(service_name="s3", region_name=region)

In [136]:
# Set path to buckets (raw data and new personal bucket)
data_location = 's3://{}'.format(bucket_name)
personal_s3 = "s3://{}/london_data/csv_files".format(personal_bucket)

In [96]:
# Check if connection was successfully established
try:
    bucket_contents = s3.list_objects(Bucket = bucket_name)["Contents"]
    print("Connected to bucket successfully!\n")
    #print("Files in S3 bucket:")
    #for i in bucket_contents:
        #print(i["Key"])
except:
    # If connection failed, print error message
    print("##### ERROR #####")
    print("Could NOT connect to bucket: {} ({})".format(bucket_name, data_location))

Connected to bucket successfully!



In [104]:
# Create Unique Paths for Each CSV File
hourly_market_data_path = personal_s3 + "/Hourly_European_Market_Data.csv"
london_energy_path = personal_s3 + "/London_Energy.csv"
london_weather_path = personal_s3 + "/London_Weather_1979_2021.csv"

# Copy Raw CSV Files to Personal S3 Bucket

In [102]:
!aws s3 cp --recursive $data_location/ $personal_s3/ --exclude "*" --include "Hourly_European_Market_Data.csv"
!aws s3 cp --recursive $data_location/ $personal_s3/ --exclude "*" --include "London_Energy.csv"
!aws s3 cp --recursive $data_location/ $personal_s3/ --exclude "*" --include "London_Weather_1979_2021.csv"

copy: s3://ads-508-spring2023-team3/Hourly_European_Market_Data.csv to s3://sagemaker-us-east-1-510267762309/london_data/csv_files/Hourly_European_Market_Data.csv
copy: s3://ads-508-spring2023-team3/London_Energy.csv to s3://sagemaker-us-east-1-510267762309/london_data/csv_files/London_Energy.csv
copy: s3://ads-508-spring2023-team3/London_Weather_1979_2021.csv to s3://sagemaker-us-east-1-510267762309/london_data/csv_files/London_Weather_1979_2021.csv


In [103]:
# Confirm that CSV files were copied to personal S3 bucket
!aws s3 ls $personal_s3/

2023-03-15 06:05:34  105185070 Hourly_European_Market_Data.csv
2023-03-15 06:05:36   95649585 London_Energy.csv
2023-03-15 06:05:38     814426 London_Weather_1979_2021.csv


# Create Athena Database

In [14]:
database_name = "london_data"

In [137]:
# Create a staging S3 directory - a temporary directory for querying
stage_dir = "s3://{}/athena/staging".format(personal_bucket)

In [138]:
# Connect to Staging Directory
try:
    pyathena_conn = connect(
        region_name = region, 
        s3_staging_dir = stage_dir)
    print("Connected to S3 staging directory!")
except:
    print("##### ERROR #####")
    print("##### Could NOT connect to S3 staging directory #####")

Connected to S3 staging directory!


In [139]:
# Create new Database - Print an error if there is a failure
try:
    sql_statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
    pd.read_sql(sql_statement, pyathena_conn)
    print("Database {} succesfully created!".format(database_name))
    sql_statement = "SHOW DATABASES"
    df_show = pd.read_sql(sql_statement, pyathena_conn)
    print(df_show.head(5))
except:
    print("##### ERROR #####")
    print("##### Could NOT create database #####")


Database london_data succesfully created!
  database_name
0       default
1        dsoaws
2   london_data


## Create Tables from CSV files
- Hourly_European_Market_Data.csv hourly_market_data_path 
- London_Energy.csv london_energy_path
- London_Weather_1979_2021.csv london_weather_path

In [140]:
# SQL Create Table State from Hourly Market Data CSV
table_name = "hourly_european_market_data_csv"

csv_table_statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        id int,
        fecha date,
        hora int,
        sistema	string,
        bandera	string,
        precio float,
        tipo_moneda	string,
        origen_dato	string,
        fecha_actualizacion	timestamp
    )
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LINES TERMINATED BY '\n'
    LOCATION '{}'
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name, hourly_market_data_path)

# Execute Create Table Statement
pd.read_sql(csv_table_statement, pyathena_conn)

""


In [141]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, pyathena_conn)
df_show.head(5)

,tab_name
0,hourly_european_market_data_csv


In [142]:
query = """
    SELECT *
    FROM {}.{}
    LIMIT 3
    """.format(database_name, "hourly_european_market_data_csv")

pd.read_sql(query, pyathena_conn)

,id,fecha,hora,sistema,bandera,precio,tipo_moneda,origen_dato,fecha_actualizacion


- Ingest raw data to Athena and trim the data to all be within the same time frame
- Export the data back out to S3

# Release Resources

In [26]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [27]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>